# Basic JUMP data access

This is a tutorial on how to access profiles from the [JUMP Cell
Painting datasets](https://github.com/jump-cellpainting/datasets). We
will use polars to fetch the data frames lazily, with the help of `s3fs`
and `pyarrow`. We prefer lazy loading because the data can be too big to
be handled in memory.

In [2]:
import polars as pl
from pyarrow.dataset import dataset
from s3fs import S3FileSystem

The shapes of the available datasets are:

1.  `cpg0016-jump[crispr]`: CRISPR knockouts genetic perturbations.
2.  `cpg0016-jump[orf]`: Overexpression genetic perturbations.
3.  `cpg0016-jump[compound]`: Chemical perturbations.

Their explicit location is determined by the transformations that
produce the datasets. The aws paths of the dataframes are built from a
prefix below:

In [4]:
_PREFIX = (
    "s3://cellpainting-gallery/cpg0016-jump-assembled/source_all/workspace/profiles"
)
_RECIPE = "jump-profiling-recipe_2024_a917fa7"

transforms = (
    (
        "CRISPR",
        "profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected",
    ),
    ("ORF", "profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony"),
    ("COMPOUND", "profiles_var_mad_int_featselect_harmony"),
)

filepaths = {
    dset: f"{_PREFIX}/{_RECIPE}/{dset}/{transform}/profiles.parquet"
    for dset, transform in transforms
}

We use a S3FileSystem to avoid the need of credentials.

In [5]:
def lazy_load(path: str) -> pl.LazyFrame:
    fs = S3FileSystem(anon=True)
    myds = dataset(path, filesystem=fs)
    df = pl.scan_pyarrow_dataset(myds)
    return df

We will lazy-load the dataframes and print the number of rows and
columns

In [6]:
info = {k: [] for k in ("dataset", "#rows", "#cols", "#Metadata cols", "Size (MB)")}
for name, path in filepaths.items():
    data = lazy_load(path)
    n_rows = data.select(pl.count()).collect().item()
    metadata_cols = data.select(pl.col("^Metadata.*$")).columns
    n_cols = data.width
    n_meta_cols = len(metadata_cols)
    estimated_size = int(round(4.03 * n_rows * n_cols / 1e6, 0))  # B -> MB
    for k, v in zip(info.keys(), (name, n_rows, n_cols, n_meta_cols, estimated_size)):
        info[k].append(v)

pl.DataFrame(info)

dataset,#rows,#cols,#Metadata cols,Size (MB)
str,i64,i64,i64,i64
"""CRISPR""",51185,3677,4,758
"""ORF""",81663,3677,4,1210
"""COMPOUND""",804844,3677,4,11926


Let us now focus on the `crispr` dataset and use a regex to select the
metadata columns. We will then sample rows and display the overview.
Note that the collect() method enforces loading some data into memory.

In [7]:
data = lazy_load(filepaths["CRISPR"])
data.select(pl.col("^Metadata.*$").sample(n=5, seed=1)).collect()

Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_JCP2022
str,str,str,str
"""source_13""","""CP-CC9-R1-06""","""M07""","""JCP2022_806374…"
"""source_13""","""CP-CC9-R1-28""","""B03""","""JCP2022_800001…"
"""source_13""","""CP-CC9-R2-23""","""P20""","""JCP2022_802185…"
"""source_13""","""CP-CC9-R3-15""","""J15""","""JCP2022_800322…"
"""source_13""","""CP-CC9-R6-28""","""O23""","""JCP2022_800002…"


The following line excludes the metadata columns:

In [8]:
data_only = data.select(pl.all().exclude("^Metadata.*$").sample(n=5, seed=1)).collect()
data_only

Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EquivalentDiameter,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_MajorAxisLength,Cells_AreaShape_MaxFeretDiameter,Cells_AreaShape_MaximumRadius,Cells_AreaShape_MeanRadius,Cells_AreaShape_MedianRadius,Cells_AreaShape_MinFeretDiameter,Cells_AreaShape_MinorAxisLength,Cells_AreaShape_Orientation,Cells_AreaShape_Perimeter,Cells_AreaShape_Solidity,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_4_2,Cells_AreaShape_Zernike_4_4,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_5_3,Cells_AreaShape_Zernike_5_5,Cells_AreaShape_Zernike_6_0,…,Nuclei_Texture_Variance_DNA_5_03_256,Nuclei_Texture_Variance_ER_10_00_256,Nuclei_Texture_Variance_ER_10_01_256,Nuclei_Texture_Variance_ER_10_02_256,Nuclei_Texture_Variance_ER_10_03_256,Nuclei_Texture_Variance_ER_3_00_256,Nuclei_Texture_Variance_ER_3_01_256,Nuclei_Texture_Variance_ER_3_02_256,Nuclei_Texture_Variance_ER_3_03_256,Nuclei_Texture_Variance_ER_5_00_256,Nuclei_Texture_Variance_ER_5_01_256,Nuclei_Texture_Variance_ER_5_02_256,Nuclei_Texture_Variance_ER_5_03_256,Nuclei_Texture_Variance_Mito_10_00_256,Nuclei_Texture_Variance_Mito_10_01_256,Nuclei_Texture_Variance_Mito_10_02_256,Nuclei_Texture_Variance_Mito_10_03_256,Nuclei_Texture_Variance_Mito_3_00_256,Nuclei_Texture_Variance_Mito_3_01_256,Nuclei_Texture_Variance_Mito_3_02_256,Nuclei_Texture_Variance_Mito_3_03_256,Nuclei_Texture_Variance_Mito_5_00_256,Nuclei_Texture_Variance_Mito_5_01_256,Nuclei_Texture_Variance_Mito_5_02_256,Nuclei_Texture_Variance_Mito_5_03_256,Nuclei_Texture_Variance_RNA_10_00_256,Nuclei_Texture_Variance_RNA_10_01_256,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
3437.205322,6777.026367,655.655762,553.805725,574.797363,475.20105,614.663025,513.953979,2.809995,0.766263,63.48436,0.934583,0.5282,0.393083,90.885178,96.299362,21.411591,7.525086,6.554964,57.310593,52.115498,0.739814,330.359772,0.785753,0.455676,0.056063,0.157158,0.05564,0.033548,0.019762,0.021171,0.03315,0.021444,0.015374,0.016197,0.010043,0.010896,…,0.287671,2.316995,2.255697,2.31177,2.281517,2.137642,2.17198,2.135844,2.171006,2.198508,2.268348,2.196704,2.264076,8.106384,8.647932,8.149483,8.731106,8.056567,7.904062,8.053588,7.903239,7.893166,7.876201,7.89196,7.928691,5.882857,5.725977,5.863044,5.74948,5.372434,5.473141,5.380401,5.467793,5.536143,5.733851,5.548314,5.717349
2723.384521,5158.307617,677.795349,557.942017,606.970093,488.857605,641.975159,522.861877,2.561128,0.741647,56.989681,0.950307,0.548386,null,78.137085,83.441475,19.805571,6.985418,6.062629,51.839073,47.675426,-2.123358,286.608917,0.801257,0.485367,0.056452,0.164563,0.055538,0.031914,0.020555,0.020632,0.031065,0.021271,0.014857,0.015759,0.010461,0.01096,…,0.425449,3.520926,3.387909,3.527264,3.404135,3.201259,3.272938,3.202795,3.268549,3.315624,3.447717,3.323368,3.439064,7.47143,8.032343,7.453537,8.07777,7.264492,7.104119,7.227602,7.152196,7.132525,7.

Finally, we can convert this to `pandas` if we want to perform analyses
with that tool. Keep in mind that this loads the entire dataframe into
memory.

In [7]:
data_only.to_pandas()

5 rows × 3673 columns